In [1]:
import ray
ray.util.connect("127.0.0.1:10001")

{'num_clients': 1,
 'python_version': '3.7.7',
 'ray_version': '2.0.0.dev0',
 'ray_commit': '5247c0a5b8740c6b6ba0156a4c772947703a42a5',
 'protocol_version': '2021-04-19'}

In [3]:
a="b"

In [2]:
import pandas as pd
import numpy as np
from bidict import bidict
from scipy.sparse import coo_matrix,csr_matrix
import implicit
from ray import tune
from implicit import evaluation
from dsrecommender.trainer import train_test_split

def map_ids(row, mapper) -> int:
    return mapper[row]

def load_data(path,frac=0.04):
    df = pd.read_csv(path,
                 header=0,
                 names=["user","item"])
    df["item"] = df.item.str.split('|')
    df = df.explode("item")
    df=df.reset_index().drop("index",axis=1)
    return df.sample(frac=frac)

In [3]:
data = load_data("s3://unext-datascience-prod/jobs/ippanreco/user_watched_sakuhins.csv.gz")

In [4]:
usersdict = bidict({user:i for i,user in enumerate(data.user.unique())})
itemsdict = bidict({item:i for i,item in enumerate(data.item.unique())})
print('Sparsity: {:4.3f}%'.format(float(data.shape[0]) / float(len(usersdict)*len(itemsdict)) * 100))

Sparsity: 0.008%


In [5]:
users=data.user.apply(map_ids,args=[usersdict]).to_numpy()
items=data.item.apply(map_ids,args=[itemsdict]).to_numpy()
ratings = np.ones(len(items))
matrix = coo_matrix((ratings, (users, items))).tocsr()

In [7]:
K=2
train_only_size=0.75
random_state = np.random.RandomState(42)
train, test = train_test_split(matrix,K,train_only_size,random_state)
tunedata = {"train":train,"test":test}

In [12]:
def train_bpr(config,traintest=None):
    import implicit
    from implicit import evaluation
    
    model = implicit.bpr.BayesianPersonalizedRanking(**config)
    train = traintest["train"].copy()
    test = traintest["test"].copy()
    model.fit(train.T)
    auc = evaluation.AUC_at_k(model,train,test)
    patk = evaluation.precision_at_k(model,train,test)
    tune.report(auc=auc,patk=patk)

In [13]:
analysis = tune.run(
    tune.with_parameters(train_bpr, traintest=tunedata),
    metric="patk",
    mode="max",
    num_samples=5,
    config={
        "factors": tune.grid_search([20, 50, 100])
    }
)
print("Best hyperparameters found were: ", analysis.best_config)

(pid=1309) == Status ==
(pid=1309) Memory usage on this node: 6.6/14.7 GiB
(pid=1309) Using FIFO scheduling algorithm.
(pid=1309) Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/28.0 GiB heap, 0.0/13.72 GiB objects (0.0/3.0 bar, 0.0/3.0 foo)
(pid=1309) Result logdir: /home/ray/ray_results/train_bpr_2021-05-18_11-54-33
(pid=1309) Number of trials: 15/15 (15 PENDING)
(pid=1309) +-----------------------+----------+-------+-----------+
(pid=1309) | Trial name            | status   | loc   |   factors |
(pid=1309) |-----------------------+----------+-------+-----------|
(pid=1309) | train_bpr_5fc9d_00000 | PENDING  |       |        20 |
(pid=1309) | train_bpr_5fc9d_00001 | PENDING  |       |        50 |
(pid=1309) | train_bpr_5fc9d_00002 | PENDING  |       |       100 |
(pid=1309) | train_bpr_5fc9d_00003 | PENDING  |       |        20 |
(pid=1309) | train_bpr_5fc9d_00004 | PENDING  |       |        50 |
(pid=1309) | train_bpr_5fc9d_00005 | PENDING  |       |       100 |
(pid=1309) | train_bpr_

  3%|▎         | 3/100 [00:03<01:37,  1.00s/it, correct=50.12%, skipped=0.13%]


(pid=1309) == Status ==
(pid=1309) Memory usage on this node: 7.5/14.7 GiB
(pid=1309) Using FIFO scheduling algorithm.
(pid=1309) Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/28.0 GiB heap, 0.0/13.72 GiB objects (0.0/3.0 bar, 0.0/3.0 foo)
(pid=1309) Result logdir: /home/ray/ray_results/train_bpr_2021-05-18_11-54-33
(pid=1309) Number of trials: 15/15 (7 PENDING, 8 RUNNING)
(pid=1309) +-----------------------+----------+-------+-----------+
(pid=1309) | Trial name            | status   | loc   |   factors |
(pid=1309) |-----------------------+----------+-------+-----------|
(pid=1309) | train_bpr_5fc9d_00000 | RUNNING  |       |        20 |
(pid=1309) | train_bpr_5fc9d_00001 | RUNNING  |       |        50 |
(pid=1309) | train_bpr_5fc9d_00002 | RUNNING  |       |       100 |
(pid=1309) | train_bpr_5fc9d_00003 | RUNNING  |       |        20 |
(pid=1309) | train_bpr_5fc9d_00004 | RUNNING  |       |        50 |
(pid=1309) | train_bpr_5fc9d_00005 | RUNNING  |       |       100 |
(pid=1309) 

 31%|███▏      | 312402/993760 [00:37<00:44, 15353.38it/s]


(pid=1309) Result for train_bpr_5fc9d_00006:
(pid=1309)   auc: 0.5034324519777387
(pid=1309)   date: 2021-05-17_19-57-19
(pid=1309)   done: false
(pid=1309)   experiment_id: e54645e53c1e49c086ccd0ac9437f608
(pid=1309)   hostname: recommender-recommender-ray-worker-k4mqm
(pid=1309)   iterations_since_restore: 1
(pid=1309)   node_ip: 10.3.8.8
(pid=1309)   patk: 0.007255139056831923
(pid=1309)   pid: 495
(pid=1309)   time_since_restore: 164.09434723854065
(pid=1309)   time_this_iter_s: 164.09434723854065
(pid=1309)   time_total_s: 164.09434723854065
(pid=1309)   timestamp: 1621306639
(pid=1309)   timesteps_since_restore: 0
(pid=1309)   training_iteration: 1
(pid=1309)   trial_id: 5fc9d_00006
(pid=1309)   
(pid=1309) == Status ==
(pid=1309) Memory usage on this node: 7.4/14.7 GiB
(pid=1309) Using FIFO scheduling algorithm.
(pid=1309) Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/28.0 GiB heap, 0.0/13.72 GiB objects (0.0/3.0 bar, 0.0/3.0 foo)
(pid=1309) Current best trial: 5fc9d_00006 with

...

(pid=1309) Result for train_bpr_5fc9d_00008:
(pid=1309)   auc: 0.5038652163294629
(pid=1309)   date: 2021-05-17_20-00-57
(pid=1309)   done: false
(pid=1309)   experiment_id: 2173df5b34d84741b3d26dc727151c4c
(pid=1309)   hostname: recommender-recommender-ray-worker-lsplz
(pid=1309)   iterations_since_restore: 1
(pid=1309)   node_ip: 10.3.12.4
(pid=1309)   patk: 0.008120809057931186
(pid=1309)   pid: 181
(pid=1309)   time_since_restore: 218.2652280330658
(pid=1309)   time_this_iter_s: 218.2652280330658
(pid=1309)   time_total_s: 218.2652280330658
(pid=1309)   timestamp: 1621306857
(pid=1309)   timesteps_since_restore: 0
(pid=1309)   training_iteration: 1
(pid=1309)   trial_id: 5fc9d_00008
(pid=1309)   
(pid=1309) == Status ==
(pid=1309) Memory usage on this node: 7.1/14.7 GiB
(pid=1309) Using FIFO scheduling algorithm.
(pid=1309) Resources requested: 2.0/10 CPUs, 0/0 GPUs, 0.0/35.0 GiB heap, 0.0/17.31 GiB objects (0.0/4.0 foo, 0.0/4.0 bar)
(pid=1309) Current best trial: 5fc9d_00005 with 

 90%|████████▉ | 890752/993760 [00:47<00:00, 184972.64it/s]


(pid=1309) Result for train_bpr_5fc9d_00014:
(pid=1309)   auc: 0.5038102966723169
(pid=1309)   date: 2021-05-18_12-01-29
(pid=1309)   done: false
(pid=1309)   experiment_id: 542fc4ff77df4e1fbe195fe425f400ab
(pid=1309)   hostname: recommender-recommender-ray-head-749kz
(pid=1309)   iterations_since_restore: 1
(pid=1309)   node_ip: 10.3.9.2
(pid=1309)   patk: 0.008010882708585248
(pid=1309)   pid: 2407
(pid=1309)   time_since_restore: 198.84862446784973
(pid=1309)   time_this_iter_s: 198.84862446784973
(pid=1309)   time_total_s: 198.84862446784973
(pid=1309)   timestamp: 1621306889
(pid=1309)   timesteps_since_restore: 0
(pid=1309)   training_iteration: 1
(pid=1309)   trial_id: 5fc9d_00014
(pid=1309)   
(pid=1309) == Status ==
(pid=1309) Memory usage on this node: 6.7/14.7 GiB
(pid=1309) Using FIFO scheduling algorithm.
(pid=1309) Resources requested: 1.0/10 CPUs, 0/0 GPUs, 0.0/35.0 GiB heap, 0.0/17.31 GiB objects (0.0/4.0 bar, 0.0/4.0 foo)
(pid=1309) Current best trial: 5fc9d_00005 with

100%|██████████| 993760/993760 [00:48<00:00, 20692.41it/s] 
(pid=1309) 2021-05-18 12:01:29,855	INFO tune.py:549 -- Total run time: 416.31 seconds (416.17 seconds for the tuning loop).


Best hyperparameters found were:  {'factors': 100}


In [ ]:
!tensorboard --logdir ~/ray_results

 47%|████▋     | 469104/993872 [01:09<01:39, 5249.13it/s]


TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784



 25%|██▌       | 249273/993872 [00:50<02:39, 4671.56it/s]


Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.5.0 at http://localhost:6006/ (Press CTRL+C to quit)


 94%|█████████▍| 933714/993872 [01:31<00:00, 122833.76it/s]


...

 86%|████████▌ | 853889/993872 [01:09<00:01, 130086.94it/s]


(pid=782) Result for _train_b70f3_00011:
(pid=782)   auc: 0.5038220121639059
(pid=782)   date: 2021-05-18_17-27-34
(pid=782)   done: false
(pid=782)   experiment_id: 4cb29b5368764d04a855bb9e614929cf
(pid=782)   hostname: recommender-recommender-ray-head-lvj9s
(pid=782)   iterations_since_restore: 1
(pid=782)   node_ip: 10.3.9.5
(pid=782)   patk: 0.008034729878721059
(pid=782)   pid: 924
(pid=782)   time_since_restore: 314.72639060020447
(pid=782)   time_this_iter_s: 314.72639060020447
(pid=782)   time_total_s: 314.72639060020447
(pid=782)   timestamp: 1621326454
(pid=782)   timesteps_since_restore: 0
(pid=782)   training_iteration: 1
(pid=782)   trial_id: b70f3_00011
(pid=782)   
(pid=782) == Status ==
(pid=782) Memory usage on this node: 6.8/14.7 GiB
(pid=782) Using FIFO scheduling algorithm.
(pid=782) Resources requested: 1.0/10 CPUs, 0/0 GPUs, 0.0/35.0 GiB heap, 0.0/17.31 GiB objects (0.0/4.0 foo, 0.0/4.0 bar)
(pid=782) Current best trial: b70f3_00002 with patk=0.008227673649393604 

100%|██████████| 993872/993872 [01:09<00:00, 14287.75it/s] 


(pid=782) Result for _train_b70f3_00011:
(pid=782)   auc: 0.5038220121639059
(pid=782)   date: 2021-05-18_17-27-34
(pid=782)   done: true
(pid=782)   experiment_id: 4cb29b5368764d04a855bb9e614929cf
(pid=782)   experiment_tag: 11_factors=100
(pid=782)   hostname: recommender-recommender-ray-head-lvj9s
(pid=782)   iterations_since_restore: 1
(pid=782)   node_ip: 10.3.9.5
(pid=782)   patk: 0.008034729878721059
(pid=782)   pid: 924
(pid=782)   time_since_restore: 314.72639060020447
(pid=782)   time_this_iter_s: 314.72639060020447
(pid=782)   time_total_s: 314.72639060020447
(pid=782)   timestamp: 1621326454
(pid=782)   timesteps_since_restore: 0
(pid=782)   training_iteration: 1
(pid=782)   trial_id: b70f3_00011
(pid=782)   
(pid=782) == Status ==
(pid=782) Memory usage on this node: 6.8/14.7 GiB
(pid=782) Using FIFO scheduling algorithm.
(pid=782) Resources requested: 0/10 CPUs, 0/0 GPUs, 0.0/35.0 GiB heap, 0.0/17.31 GiB objects (0.0/4.0 bar, 0.0/4.0 foo)
(pid=782) Current best trial: b70

(pid=782) 2021-05-18 17:27:34,671	INFO tune.py:549 -- Total run time: 507.38 seconds (507.08 seconds for the tuning loop).
